Python’s asyncio module is used for asynchronous programming (async / await).
Normally, you run async code like this:

That works fine in a normal Python script.

👉 But in some environments (like Jupyter Notebook, IPython, or Google Colab), there’s already an event loop running in the background.
If you try to call asyncio.run() there, you’ll get: below error after running the below code.

In [ ]:
import asyncio

async def main():
  print("Hello")
  await asyncio.sleep(1)
  print("World")
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

🔹 What nest_asyncio does

nest_asyncio is a small library that patches Python’s asyncio so you can nest event loops inside each other.

nest_asyncio.apply() modifies (monkey-patches) the default event loop behavior.

After applying it, you can safely call async functions (and even start new event loops) inside an environment that already has one running.

In [ ]:
import nest_asyncio
import asyncio

# Fix event loop so we can nest it
nest_asyncio.apply()

async def greet():
    print("Hello")
    await asyncio.sleep(1)
    print("World")

# ✅ Now works fine even in Jupyter
asyncio.run(greet())

Hello
World


✅ In short:
nest_asyncio.apply() patches asyncio so you can re-enter or nest event loops. It’s mainly used in interactive environments (like Jupyter) where an event loop is already running, so you can still run asyncio.run() or await things normally.



---



🔹 1. The problem asyncio solves

Normally, Python executes code synchronously — one line at a time.

Example:

In [ ]:
import time

def task1():
    print("Start task1")
    time.sleep(2)   # Blocks program for 2 seconds
    print("End task1")

def task2():
    print("Start task2")
    time.sleep(2)   # Blocks program for 2 seconds
    print("End task2")

task1()
task2()

Start task1
End task1
Start task2
End task2


➡️ Both tasks block each other because time.sleep() stops everything.

🔹 2. Enter asyncio

asyncio is Python’s library for asynchronous programming.

It lets you write code that can pause at certain points (like waiting for I/O, network, or sleep)

and allow other tasks to run in the meantime.

This doesn’t create new threads or processes → it’s still single-threaded, but it switches between tasks when one is waiting.

Think of it like:

Synchronous: A chef cooks dish A completely, then starts dish B.

Asynchronous: While dish A is in the oven (waiting), the chef starts cooking dish B.

Example-1

In [ ]:
import asyncio
import time

async def async_task(name, duration):
  """An asynchronous task that sleeps for a given duration."""
  print(f"Start {name}")
  await asyncio.sleep(duration)   # Non-blocking sleep
  print(f"End {name}")

async def main():
  """Runs multiple asynchronous tasks concurrently."""
  print("Starting asynchronous tasks...")
  # Run multiple tasks at the same time
  await asyncio.gather(
      async_task("Task 1", 3),
      async_task("Task 2", 2),
      async_task("Task 3", 4),
      async_task("Task 4", 1)
  )
  print("All asynchronous tasks finished.")

# Use nest_asyncio to allow asyncio.run() in environments with a running event loop
import nest_asyncio
nest_asyncio.apply()

# Run the main asynchronous function
asyncio.run(main())

Starting asynchronous tasks...
Start Task 1
Start Task 2
Start Task 3
Start Task 4
End Task 4
End Task 2
End Task 1
End Task 3
All asynchronous tasks finished.


all the run concurrently using `asyncio.gather()`.

Example-2

In [ ]:
import asyncio

async def task1():
    print("Start task1")
    await asyncio.sleep(2)   # Non-blocking sleep
    print("End task1")

async def task2():
    print("Start task2")
    await asyncio.sleep(2)
    print("End task2")

async def main():
    # Run both tasks at the same time
    await asyncio.gather(task1(), task2())

asyncio.run(main())

Start task1
Start task2
End task1
End task2


###**Key concepts in `asyncio`**

1) `async def` : Defines a coroutine (a special function that can pause/resume).

```
async def my_task():
  ...
```

2) `await` : Pauses the coroutine until the awaited task finishes.

```
await asyncio.sleep(1) # give control back to event loop
```

3) `Event Loop` : The "scheduler" that runs all async tasks.

`asyncio.run(main())`

starts the event loop, runs main(), and stops the loop when finished.

4) `asyncio.gather()` : Run multiple coroutines concurrently.

```
await asyncio.gather(task1(), task2(), task3())
```

5) Difference between `time.sleep()` and `asyncio.sleep()`

`time.sleep(2)` → blocks everything for 2 seconds.

`await asyncio.sleep(2)` → pauses this coroutine but allows other coroutines to run during that time.